<a href="https://colab.research.google.com/github/lorenz0leoncin1/tree-predictors-mushroom-classification/blob/main/FS_Tree_predictors_for_binary_classification_mushrooms.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Phase 1: Import libraries & dataset

In [201]:
# Install necessary packages
!pip3 install -U ucimlrepo
!pip3 install -U scikit-learn

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from ucimlrepo import fetch_ucirepo
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.feature_selection import SelectFromModel
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split

In [202]:
# fetch dataset
secondary_mushroom = fetch_ucirepo(id=848)

# data (as pandas dataframes)
X = secondary_mushroom.data.features
y = secondary_mushroom.data.targets

In [203]:
X.head()

,cap-diameter,cap-shape,cap-surface,cap-color,does-bruise-or-bleed,gill-attachment,gill-spacing,gill-color,stem-height,stem-width,stem-root,stem-surface,stem-color,veil-type,veil-color,has-ring,ring-type,spore-print-color,habitat,season
0,15.26,x,g,o,f,e,NaN,w,16.95,17.09,s,y,w,u,w,t,g,NaN,d,w
1,16.60,x,g,o,f,e,NaN,w,17.99,18.19,s,y,w,u,w,t,g,NaN,d,u
2,14.07,x,g,o,f,e,NaN,w,17.80,17.74,s,y,w,u,w,t,g,NaN,d,w
3,14.17,f,h,e,f,e,NaN,w,15.77,15.98,s,y,w,u,w,t,p,NaN,d,w
4,14.64,x,h,o,f,e,NaN,w,16.53,17.20,s,y,w,u,w,t,p,NaN,d,w


In [204]:
y.head()

,class
0,p
1,p
2,p
3,p
4,p


In [205]:
print(X.shape)
print(y.shape)

(61069, 20)
(61069, 1)


# Phase 2: Data pre-processing

In [206]:
# Check for missing values
missing_values = X.isnull().sum()
print("Missing values per feature:\n", missing_values)

Missing values per feature:
 cap-diameter                0
cap-shape                   0
cap-surface             14120
cap-color                   0
does-bruise-or-bleed        0
gill-attachment          9884
gill-spacing            25063
gill-color                  0
stem-height                 0
stem-width                  0
stem-root               51538
stem-surface            38124
stem-color                  0
veil-type               57892
veil-color              53656
has-ring                    0
ring-type                2471
spore-print-color       54715
habitat                     0
season                      0
dtype: int64


In [207]:
# Threshold for dropping columns with too many missing values (e.g., more than 50%)
threshold = 0.5 * len(X)  # 50% of total rows
X_dropped = X.drop(columns=[col for col in X.columns if X[col].isnull().sum() > threshold])
print("Dataset after removing columns with too many missing values:\n", X_dropped.head())

Dataset after removing columns with too many missing values:
    cap-diameter cap-shape cap-surface cap-color does-bruise-or-bleed  \
0         15.26         x           g         o                    f   
1         16.60         x           g         o                    f   
2         14.07         x           g         o                    f   
3         14.17         f           h         e                    f   
4         14.64         x           h         o                    f   

  gill-attachment gill-spacing gill-color  stem-height  stem-width stem-color  \
0               e          NaN          w        16.95       17.09          w   
1               e          NaN          w        17.99       18.19          w   
2               e          NaN          w        17.80       17.74          w   
3               e          NaN          w        15.77       15.98          w   
4               e          NaN          w        16.53       17.20          w   

  has-ring ring-ty

In [208]:
# Controllo dei valori mancanti nel nuovo DataFrame
missing_values_dropped = X_dropped.isnull().sum()
print("Missing values per feature dopo la rimozione delle colonne:\n", missing_values_dropped)

# Visualizzazione dei tipi di dato delle colonne
print("Tipi di dato delle colonne:\n", X_dropped.dtypes)


Missing values per feature dopo la rimozione delle colonne:
 cap-diameter                0
cap-shape                   0
cap-surface             14120
cap-color                   0
does-bruise-or-bleed        0
gill-attachment          9884
gill-spacing            25063
gill-color                  0
stem-height                 0
stem-width                  0
stem-color                  0
has-ring                    0
ring-type                2471
habitat                     0
season                      0
dtype: int64
Tipi di dato delle colonne:
 cap-diameter            float64
cap-shape                object
cap-surface              object
cap-color                object
does-bruise-or-bleed     object
gill-attachment          object
gill-spacing             object
gill-color               object
stem-height             float64
stem-width              float64
stem-color               object
has-ring                 object
ring-type                object
habitat                  object

In [209]:
# Visualizzazione delle prime righe del DataFrame finale
print("Prime righe del DataFrame finale:\n", X_dropped.head())


Prime righe del DataFrame finale:
    cap-diameter cap-shape cap-surface cap-color does-bruise-or-bleed  \
0         15.26         x           g         o                    f   
1         16.60         x           g         o                    f   
2         14.07         x           g         o                    f   
3         14.17         f           h         e                    f   
4         14.64         x           h         o                    f   

  gill-attachment gill-spacing gill-color  stem-height  stem-width stem-color  \
0               e          NaN          w        16.95       17.09          w   
1               e          NaN          w        17.99       18.19          w   
2               e          NaN          w        17.80       17.74          w   
3               e          NaN          w        15.77       15.98          w   
4               e          NaN          w        16.53       17.20          w   

  has-ring ring-type habitat season  
0      

In [210]:
# Fill missing values in object-type columns with 'unknown'
object_cols = X_dropped.select_dtypes(include=['object']).columns
X_dropped[object_cols] = X_dropped[object_cols].fillna('unknown')

# Check the result
missing_values_after_fill = X_dropped.isnull().sum()
print("Missing values after filling:\n", missing_values_after_fill)


Missing values after filling:
 cap-diameter            0
cap-shape               0
cap-surface             0
cap-color               0
does-bruise-or-bleed    0
gill-attachment         0
gill-spacing            0
gill-color              0
stem-height             0
stem-width              0
stem-color              0
has-ring                0
ring-type               0
habitat                 0
season                  0
dtype: int64


In [211]:
# Define categorical and numerical columns
categorical_cols = X_dropped.select_dtypes(include=['object']).columns.tolist()
numerical_cols = X_dropped.select_dtypes(include=['float64', 'int64']).columns.tolist()

# Create the preprocessor
preprocessor = ColumnTransformer(
    transformers=[
        ('num', 'passthrough', numerical_cols),  # Keep numerical columns as is
        ('cat', OneHotEncoder(sparse_output=False, handle_unknown='ignore'), categorical_cols)  # Apply OneHotEncoding to categorical columns
    ]
)


# Phase 3: Creating class Node


In [212]:
class Node:
    def __init__(self, split_criterion=None, is_leaf=False, prediction=None):
        """
        Initializes the node.
        :param split_criterion: Function that decides the split based on a feature
        :param is_leaf: Boolean flag to check if the node is a leaf
        :param prediction: Predicted value if the node is a leaf
        """
        self.split_criterion = split_criterion  # Function for splitting decision
        self.is_leaf = is_leaf  # Indicates if the node is a leaf
        self.prediction = prediction  # Predicted value if it's a leaf
        self.left_child = None  # Left child node
        self.right_child = None  # Right child node

    def apply_split(self, data_point):
        """
        Applies the split criterion to a data point.
        :param data_point: A row of data (sample) to test
        :return: True if the sample should go to the left, False if to the right
        """
        if self.split_criterion is None:
            raise ValueError("Split criterion is not defined.")

        # Apply the split function to the data point
        return self.split_criterion(data_point)

    def set_children(self, left_child, right_child):
        """
        Assigns the left and right children to the current node.
        :param left_child: Left child node
        :param right_child: Right child node
        """
        self.left_child = left_child
        self.right_child = right_child


#Phase 4: Creating Class TreePredictor

In [221]:
class TreePredictor:
    def __init__(self, split_criterion_func, max_depth=None, min_samples_split=2, min_impurity_decrease=0.0):

        """
        Initializes the decision tree.
        :param split_criterion_func: Function to evaluate the split (e.g., Gini index, entropy)
        :param max_depth: Maximum depth of the tree
        :param min_samples_split: Minimum number of samples required to split a node
        :param min_impurity_decrease: Minimum decrease in impurity required to split
        """
        self.split_criterion_func = split_criterion_func  # Function to evaluate split quality
        self.max_depth = max_depth  # Maximum depth of the tree
        self.min_samples_split = min_samples_split  # Minimum samples to split
        self.min_impurity_decrease = min_impurity_decrease  # Minimum impurity decrease for split
        self.root = None  # Root of the tree
        self.feature_names_ = None  # Initialize feature_names_ to None

    def fit(self, X, y):
        """
        Trains the tree using the provided dataset.
        :param X: Feature matrix (numpy array)
        :param y: Target vector (numpy array)
        """

        # Check if X is a DataFrame and has columns attribute
        if hasattr(X, 'columns'):
            self.feature_names_ = X.columns
        # If X is a NumPy array, generate feature names
        elif isinstance(X, np.ndarray):
            self.feature_names_ = [f"feature_{i}" for i in range(X.shape[1])]
        # Handle other data types or raise an error if necessary
        else:
            raise ValueError("Input X must be a pandas DataFrame or a NumPy array.")
        # Start building the tree from the root node
        self.root = self._grow_tree(X, y, depth=0)

    #TO DO DIFFICULT
    def _grow_tree(self, X, y, depth):
        """
        Recursively grows the decision tree.
        :param X: Feature matrix
        :param y: Target vector
        :param depth: Current depth of the tree
        :return: Node
        """
        num_samples, num_features = X.shape
        # Check stopping criteria
        if depth >= self.max_depth or num_samples < self.min_samples_split or len(set(y)) == 1:
            # Create a leaf node with the majority class
            leaf_value = self._most_common_label(y)
            return Node(is_leaf=True, prediction=leaf_value)

        # Find the best split
        best_feature, best_threshold = self._best_split(X, y)
        if best_feature is None:
            # Create a leaf node if no split is found
            leaf_value = self._most_common_label(y)
            return Node(is_leaf=True, prediction=leaf_value)

        # Split the data
        left_indices, right_indices = self._split(X[:, best_feature], best_threshold)
        left_child = self._grow_tree(X[left_indices, :], y[left_indices], depth + 1)
        right_child = self._grow_tree(X[right_indices, :], y[right_indices], depth + 1)

        # Create an internal node
        node = Node(split_criterion=lambda data_point: data_point[best_feature] <= best_threshold)  # Use best_feature (numerical index) directly
        node.set_children(left_child, right_child)
        return node

    def _split(self, feature_column, threshold):
        """
        Splits the data into left and right groups based on the threshold.
        :param feature_column: A feature column
        :param threshold: Threshold value for splitting
        :return: Indices of left and right groups
        """
        # Check if feature_column is already a dense array
        if not isinstance(feature_column, np.ndarray):
            feature_column = feature_column.toarray().flatten()  # Convert to dense array if needed

        # Perform the split
        left_indices = np.where(feature_column <= threshold)[0]
        right_indices = np.where(feature_column > threshold)[0]
        return left_indices, right_indices


    def _best_split(self, X, y):
        """
        Finds the best feature and threshold to split the data.
        :param X: Feature matrix
        :param y: Target vector
        :return: Best feature index and threshold for the split
        """
        best_feature = None
        best_threshold = None
        best_impurity = float('inf')

        # Iterate using feature indices directly
        for feature_index in range(X.shape[1]):

            feature_column = X[:, feature_index]  # Access feature column by index

            # Check if the feature column is already a dense array
            if not isinstance(feature_column, np.ndarray):
                feature_column = feature_column.toarray().flatten()  # Convert to dense array if needed

            # Possible thresholds for this feature
            thresholds = set(feature_column)
            for threshold in thresholds:
                # Split the data based on the threshold
                left_indices, right_indices = self._split(feature_column, threshold)
                if len(left_indices) == 0 or len(right_indices) == 0:
                    continue

                # Calculate impurity of the split
                impurity = self._calculate_impurity(y[left_indices], y[right_indices])
                if impurity < best_impurity:
                    best_impurity = impurity
                    best_feature = feature_index  # Store the feature index (not the name)
                    best_threshold = threshold

        return best_feature, best_threshold

    def _calculate_impurity(self, left_y, right_y):
        """
        Calculates the impurity of a split (e.g., Gini index or entropy).
        :param left_y: Target values for the left group
        :param right_y: Target values for the right group
        :return: Impurity of the split
        """
        left_weight = len(left_y) / (len(left_y) + len(right_y))
        right_weight = 1 - left_weight

        # You can use Gini index, entropy, or other criteria
        impurity = left_weight * self.split_criterion_func(left_y) + right_weight * self.split_criterion_func(right_y)
        return impurity

    def _most_common_label(self, y):
        """
        Returns the most common label in the target vector.
        Assumes y contains numeric labels (0 or 1).
        """
        # Get the unique labels and their counts
        unique_labels, counts = np.unique(y, return_counts=True)

        # Find the index of the most frequent label
        most_frequent_index = np.argmax(counts)

        # Get the actual label value using the index
        most_frequent_label = unique_labels[most_frequent_index]

        return most_frequent_label

    def predict(self, X):
        """
        Predicts the class for each sample in the dataset.
        :param X: Feature matrix (numpy array)
        :return: Predicted classes
        """
        return [self._predict_sample(x) for x in X]

    def _predict_sample(self, x):
        """
        Predicts the class for a single sample.
        :param x: Single data point (numpy array)
        :return: Predicted class
        """
        node = self.root
        while not node.is_leaf:
            if node.apply_split(x):
                node = node.left_child
            else:
                node = node.right_child
        return node.prediction


#Phase 5: Train the tree predictors adopting at least 3 reasonable criteria for the expansion of the leaves

In [222]:
def gini_index(y):
    """Calculate Gini Index for a given array of labels."""
    m = len(y)
    if m == 0:
        return 0
    p = np.bincount(y) / m
    return 1 - np.sum(p ** 2)

def entropy(y):
    """Calculate Entropy for a given array of labels."""
    m = len(y)
    if m == 0:
        return 0
    p = np.bincount(y) / m
    return -np.sum(p * np.log2(p + 1e-10))  # Add small constant to avoid log(0)

def train_and_evaluate(X_train, y_train, X_test, y_test):
    """
    Trains and evaluates the tree predictor with different criteria.
    :param X_train: Training feature matrix
    :param y_train: Training target vector
    :param X_test: Testing feature matrix
    :param y_test: Testing target vector
    """

    criteria = [gini_index, entropy]
    for criterion in criteria:
        print(f"Training with criterion: {criterion.__name__}")
        tree = TreePredictor(split_criterion_func=criterion, max_depth=5, min_samples_split=2)
        tree.fit(X_train_processed, y_train)

        # Calculate training predictions and error
        train_predictions = tree.predict(X_train_processed)
        train_error = np.mean(train_predictions != y_train)
        print(f"Training Error: {train_error:.4f}")

        # Calculate test predictions and error
        test_predictions = tree.predict(X_test_processed)
        test_error = np.mean(test_predictions != y_test)
        print(f"Test Error: {test_error:.4f}")

In [223]:
# Assuming X_dropped and y are your original data
le = LabelEncoder()
y_numeric = le.fit_transform(y.to_numpy().ravel())  # Convert to numpy array, then flatten

In [224]:
# Supponendo che X_dropped e y siano già definiti
# Crea la suddivisione del dataset in addestramento e test
X_train, X_test, y_train, y_test = train_test_split(X_dropped, y_numeric, test_size=0.2, random_state=42)

# Preprocessare i dati di addestramento e di test
X_train_processed = preprocessor.fit_transform(X_train)
X_test_processed = preprocessor.transform(X_test)

# Allenare e valutare il modello
train_and_evaluate(X_train_processed, y_train, X_test_processed, y_test)

Training with criterion: gini_index
Training Error: 0.2837
Test Error: 0.2879
Training with criterion: entropy
Training Error: 0.2856
Test Error: 0.2895


#Siamo arrivati qua, bisogna capire come fare il tuning degli iperparametri  come usarli